In [1]:
data_var = '2024-01-10'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
9863,2024-01-10,Eua Nba,00:30,Los Angeles Lakers,Toronto Raptors,1.58,2.62,235.5,1.86,2.07,-4.5,1.97,1.97,j7byR1Ih,0.632911,0.381679,0.537634,0.483092,0.014591,0.6,0.8,NaN,NaN,240.228,77.075398,0.320843,260.398,59.605089,0.228900,207.590,305.378,293.62,285.95,0.0,0.0,0.0,0.0,0.350186,0.075569,0.000000,-1.09,-0.218,-2.660550,0.611424,0.5,-0.111424,-0.67,-0.134,-12.089552,0.470828,0.3,-0.170828
9864,2024-01-10,Eua Nba,21:00,Detroit Pistons,San Antonio Spurs,2.38,1.68,237.5,1.82,2.12,3.5,1.98,1.96,n3L6hDym,0.420168,0.595238,0.549451,0.471698,0.015406,0.4,0.6,NaN,NaN,701.468,541.455743,0.771889,556.634,122.969420,0.220916,398.098,465.422,433.40,587.65,1.0,1.0,0.0,0.0,0.243830,0.107681,0.007179,-2.60,-0.520,-2.653846,0.356352,0.1,-0.256352,-2.54,-0.508,-1.338583,0.240996,0.1,-0.140996
9865,2024-01-10,Eua Nba,21:00,Indiana Pacers,Washington Wizards,1.35,3.63,249.5,1.80,2.09,-8.5,2.09,1.85,QgKAiXjg,0.740741,0.275482,0.555556,0.478469,0.016223,0.6,0.6,NaN,NaN,285.124,77.813071,0.272910,419.366,54.692388,0.130417,248.112,613.044,238.36,494.10,0.0,0.0,0.0,0.0,0.647471,0.105430,0.086145,0.52,0.104,3.365385,0.622198,0.7,0.077802,-2.38,-0.476,-5.525210,0.196854,0.1,-0.096854
9866,2024-01-10,Eua Nba,21:30,Atlanta Hawks,Philadelphia 76ers,1.69,2.36,241.5,1.81,2.07,-3.5,2.03,1.89,YJJEji6a,0.591716,0.423729,0.552486,0.483092,0.015445,0.6,0.4,NaN,NaN,236.166,54.242232,0.229678,168.078,56.102350,0.333788,236.710,206.290,277.77,194.12,0.0,0.0,0.0,0.0,0.233956,0.094767,0.050508,-3.03,-0.606,-1.138614,0.583389,0.3,-0.283389,-0.24,-0.048,-28.333333,0.549829,0.5,-0.049829
9867,2024-01-10,Eua Nba,21:30,Miami Heat,Oklahoma City Thunder,2.51,1.62,231.5,1.80,2.08,3.5,2.00,1.94,CAIIkBL5,0.398406,0.617284,0.555556,0.480769,0.015690,0.8,0.6,NaN,NaN,316.034,70.924663,0.224421,224.642,54.357089,0.241972,188.442,248.146,208.25,172.50,0.0,0.0,0.0,0.0,0.304758,0.102057,0.021536,-0.41,-0.082,-18.414634,0.585330,0.5,-0.085330,-2.05,-0.410,-1.512195,0.530733,0.5,-0.030733
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9996,2024-01-10,Eua Ncaa,22:00,Valparaiso,Southern Illinois,4.67,1.21,133.5,1.91,1.91,9.5,1.91,1.91,AR73vwb1,0.214133,0.826446,0.523560,0.523560,0.040579,0.4,0.0,NaN,NaN,252.068,128.572253,0.510070,98.738,16.396390,0.166060,247.418,0.000,341.60,124.96,1.0,0.0,0.0,0.0,0.832173,0.000000,0.000000,-3.20,-0.640,-5.734375,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9997,2024-01-10,Eua Ncaa,22:30,Troy,La Lafayette,1.69,2.22,145.5,1.80,1.95,-3.5,2.00,1.71,8beActlP,0.591716,0.450450,0.555556,0.512821,0.042166,0.8,0.8,NaN,NaN,180.032,96.302948,0.534921,170.794,57.632181,0.337437,132.930,202.824,153.78,207.48,0.0,0.0,0.0,0.0,0.191696,0.056569,0.110545,3.37,0.674,1.023739,0.000000,0.0,0.000000,-2.53,-0.506,-2.411067,0.000000,0.0,0.000000
9998,2024-01-10,Europa Wbbl Feminina,14:30,Jurmala F,Liepajas Papirs/LSSS F,9.70,1.01,132.5,1.84,1.86,18.5,1.81,1.89,WfqJ93zQ,0.103093,0.990099,0.543478,0.537634,0.093192,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,1.147480,0.007644,0.030578,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
9999,2024-01-10,Islândia Premier League,16:15,Keflavik,Tindastoll,1.48,2.53,176.5,1.81,1.87,-6.5,2.08,1.63,4WBqUwOT,0.675676,0.395257,0.

## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,00:30,Eua Nba,Los Angeles Lakers,Toronto Raptors,235.5,1.86,1.0000,Over
1,21:00,Eua Nba,Detroit Pistons,San Antonio Spurs,237.5,1.82,1.0000,Over
2,21:00,Eua Nba,Indiana Pacers,Washington Wizards,249.5,1.80,1.0000,Over
3,21:30,Eua Nba,Atlanta Hawks,Philadelphia 76ers,241.5,1.81,0.9999,Over
4,21:30,Eua Nba,Miami Heat,Oklahoma City Thunder,231.5,1.80,1.0000,Over
5,22:00,Eua Nba,Chicago Bulls,Houston Rockets,213.5,1.80,1.0000,Over
6,16:30,Europa Euroliga,Barcelona,Olympiakos,155.5,1.80,1.0000,Over
7,11:00,Rússia Liga Vtb United,Astana,Parma Perm,157.5,1.88,0.9996,Over
8,21:00,Eua Nba,Charlotte Hornets,Sacramento Kings,230.5,1.82,1.0000,Over
9,21:00,Eua Ncaa,Holy Cross,Navy,135.5,1.95,0.9871,Over
